In [1]:
from dashboard import Dashboard
ds = Dashboard(graphs=False)

Leyendo datos desde archivo guardado.


In [2]:
import yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [3]:
from src.calculations import Decadiarias
import numpy as np
import pandas as pd

In [4]:
df = ds.pp[ds.pp.index.month == ds.this_month]
# Creando dataframe de precipitación y añadiendo columna con nro de decadiaria
df.insert(0, "Decadiaria", np.nan)

df.loc[df.index.day <= 10, "Decadiaria"] = 1
df.loc[(df.index.day > 10) & (df.index.day <= 20), "Decadiaria"] = 2
df.loc[df.index.day > 20, "Decadiaria"] = 3

In [7]:
# Calculando suma de valores por decadiaria
sum_df = df.groupby(pd.Grouper("Decadiaria")).sum()

In [8]:
self_dc = pd.read_excel(ds.dec_file)
# Escogiendo decadiarias del mes actual:
dc = self_dc[self_dc["MES"] == ds.this_month]

In [10]:
# Omitiendo estaciones con normales iguales a 0
columns_to_drop = []
for i in dc.columns[2:]:
    if (dc[i] == 0).any():
        columns_to_drop.append(i)

In [12]:
# New lines
for s in columns_to_drop:
    if s not in sum_df.columns:
        print(f"{s} no se encuentra disponible en la web.")
        print(f"Excluyendo {s} de columns_to_drop")
        columns_to_drop.remove(s)

CASCABAMBA no se encuentra disponible en la web.
Excluyendo CASCABAMBA de columns_to_drop


In [14]:
dc.drop(columns=columns_to_drop, inplace=True)

In [16]:
sum_df.drop(columns=columns_to_drop, inplace=True)

In [17]:
ee = sum_df.columns
idd = sum_df.columns.isin(dc.columns)
sum_df_f = sum_df[ee[idd]]  # filtro de estaciones con decadiarias
dc_f = dc[ee[idd]]  # filtro en archivo de decadiarias

    #return sum_df_f, dc_f

In [9]:
dec_data = Decadiarias(ds.pp, ds.this_month, ds.file, ds.dec_file)

KeyError: "['CASCABAMBA'] not found in axis"

In [11]:
df = ds.pp[ds.pp.index.month == ds.this_month]
df.insert(0, "Decadiaria", np.nan)

df.loc[df.index.day <= 10, "Decadiaria"] = 1
df.loc[(df.index.day > 10) & (df.index.day <= 20), "Decadiaria"] = 2
df.loc[df.index.day > 20, "Decadiaria"] = 3

sum_df = df.groupby(pd.Grouper("Decadiaria")).sum()

In [20]:
dc = dec_data.dc[dec_data.dc["MES"] == ds.this_month]

In [ ]:
columns_to_drop = []
for i in dc.columns[2:]:
    if (dc[i] == 0).any():
        columns_to_drop.append(i)

dc.drop(columns=columns_to_drop, inplace=True)
sum_df.drop(columns=columns_to_drop, inplace=True)

In [25]:
ee = sum_df.columns
idd = sum_df.columns.isin(dc.columns)
sum_df_f = sum_df[ee[idd]]  # filtro de estaciones con decadiarias
dc_f = dc[ee[idd]]  # filtro en archivo de decadiarias

In [32]:
shape = dc_f.shape
new_array = np.zeros(shape)
new_array[:sum_df_f.shape[0], :] = sum_df_f

In [42]:
dec = (((new_array / dc_f) - 1) * 100).round(1)

In [46]:
idx = ["1ra-Dec", "2da-Dec", "3ra-Dec"]
dec_df = dec.T.copy()
dec_df.columns = idx

In [62]:
val = [-100, -60, -30, -15, 15, 30, 60, 100, 200, 400, 800, np.inf]
cols = [
    "#c2523c",
    "#e99519",
    "#fbea06",
    "#ffffff",
    "#b8f501",
    "#40e300",
    "#1ab04d",
    "#19875a",
    "#089ead",
    "#2b61a9",
    "#0b2c7b",
]

def assign_color(value):
    if value == -100:
        return "#c2523c"
    else:
        return pd.cut([value], bins=val, labels=cols)[0]

# Aplicar la función a las columnas correspondientes
dec_df.loc[:, "C1"] = dec_df.loc[:, "1ra-Dec"].apply(assign_color)
dec_df.loc[:, "C2"] = dec_df.loc[:, "2da-Dec"].apply(assign_color)
dec_df.loc[:, "C3"] = dec_df.loc[:, "3ra-Dec"].apply(assign_color)

In [64]:
from scripts.calculations import Utils

In [ ]:
dec_df = Utils(dec_df, ds.file).add_lat_lon()

In [69]:
dec = dec.rename_axis("Estaciones").reset_index()

In [70]:
dec

,Estaciones,Lat,Lon,1ra-Dec,2da-Dec,3ra-Dec,C1,C2,C3
0,CAJABAMBA,-7.62166,-78.05131,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c
1,ASUNCION,-7.32604,-78.51582,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c
2,GRANJA PORCON,-7.03753,-78.63340,-37.0,-90.3,-100.0,#e99519,#c2523c,#c2523c
3,AUGUSTO WEBERBAUER,-7.16750,-78.49309,-99.7,-99.7,-100.0,#c2523c,#c2523c,#c2523c
4,COSPAN,-7.42857,-78.54106,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c
5,LA ENCAÑADA,-7.12327,-78.33314,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c
6,JESUS,-7.24570,-78.38841,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c
7,MAGDALENA,-7.25346,-78.65261,-25.0,-100.0,-100.0,#fbea06,#c2523c,#c2523c
8,NAMORA,-7.20060,-78.32782,-100.0,-63.2,-100.0,#c2523c,#c2523c,#c2523c
9,SAN JUAN,-7.29756,-78.49106,-100.0,-100.0,-100.0,#c2523c,#c2523c,#c2523c


In [4]:
import pandas as pd
from scripts.calculations import Utils

In [2]:
last_data = {
    "ESTACIONES": ds.mn.columns,
    "TMIN": ds.mn[(ds.mn.index.day == ds.day) & (ds.mn.index.month == ds.this_month)].values[-1].tolist(),
    "TMAX": ds.mx[(ds.mx.index.day == ds.yday) & (ds.mx.index.month == ds.this_month)].values[-1].tolist(),
    "PP": ds.pp[(ds.pp.index.day == ds.yday) & (ds.pp.index.month == ds.this_month)].values[-1].tolist(),
}

In [5]:
last_data_df = pd.DataFrame(last_data)
last_data_df.set_index("ESTACIONES", inplace=True)
last_data_df = Utils(last_data_df, ds.file).add_lat_lon()


In [6]:
ds.last_data_df

,Lat,Lon,TMIN,TMAX,PP
ESTACIONES,,,,,
CACHACHI,-7.45106,-78.26855,6.2,17.2,0.0
CAJABAMBA,-7.62166,-78.05131,8.9,22.7,0.0
ASUNCION,-7.32604,-78.51582,12.8,23.6,0.0
GRANJA PORCON,-7.03753,-78.63340,2.3,18.7,0.0
AUGUSTO WEBERBAUER,-7.16750,-78.49309,5.8,22.2,0.0
COSPAN,-7.42857,-78.54106,NaN,NaN,NaN
LA ENCAÑADA,-7.12327,-78.33314,2.3,19.6,0.0
JESUS,-7.24570,-78.38841,6.8,28.2,0.0
MAGDALENA,-7.25346,-78.65261,17.0,29.0,0.0


In [14]:
import plotly.graph_objects as go
import plotly.express as px

Meteorograma

In [10]:
df = ds.last_data_df.reset_index()

In [15]:
tmx = go.Scatter(
            x=df["ESTACIONES"],
            y=df["TMAX"],
            name="Tmax de ayer (°C)",
            mode="markers+text",
            marker_symbol="triangle-up",
            marker_size=10,
            yaxis="y2",
            line={"color": "Red"},
            text=df["TMAX"].astype(str).tolist(),
            textposition="top center",
            textfont={"size": 14, "color": "crimson"},
        )
tmn = go.Scatter(
    x=df["ESTACIONES"],
    y=df["TMIN"],
    name="Tmin de hoy (°C)",
    yaxis="y2",
    line={"color": "Blue"},
    mode="markers+text",
    marker_symbol="square",
    marker_size=8,
    text=df["TMIN"].astype(str).tolist(),
    textposition="top center",
    textfont={"size": 14, "color": "blue"},
)
ppp = go.Bar(
    x=df["ESTACIONES"],
    y=df["PP"],
    name="PP de ayer (mm/día)",
    marker_color="yellowgreen",
    opacity=0.6,
    text=df["PP"].astype(str).tolist(),
    textposition="auto",
    textfont={"size": 14, "color": "darkgreen"},
)
layout = go.Layout(
    xaxis={
        "title_font": {"color": "rgba(100,143,240,255)"},
        "domain": [0, 0.94],
        "linewidth": 0.5,
        "linecolor": "rgba(196,198,206,255)",
        "showline": True,
        "showgrid": True,
        "gridwidth": 1,
        "gridcolor": "#e3e2e1",
    },
    yaxis={
        "title": "Precipitacion (mm/día)",
        "title_font": {"color": "rgba(100,143,240,255)"},
        "tickfont": {"color": "rgba(100,143,240,255)"},
        "domain": [0, 0.94],
        "side": "right",
        "showgrid": False,
        "gridwidth": 0.5,
        "gridcolor": "#e3e2e1",
    },
    yaxis2={
        "title": "Temperatura (°C)",
        "title_font": {"color": "rgba(100,143,240,255)"},
        "tickfont": {"color": "rgba(100,143,240,255)"},
        "anchor": "x",
        "overlaying": "y",
        "side": "left",
        "showgrid": True,
        "gridwidth": 0.5,
        "gridcolor": "#e3e2e1",
        "dtick": 3,
    },
    margin={"l": 50, "r": 50, "b": 20, "t": 40, "pad": 4},
    plot_bgcolor="rgb(255, 255, 255)",
    paper_bgcolor="rgb(255, 255, 255)",
    showlegend=True,
    legend={
        "y": 1.1,
        "x": 0.3,
        "yanchor": "top",
        "bordercolor": "rgba(244,244,244,255)",
        "borderwidth": 1,
        "orientation": "h",
    },
    xaxis_tickangle=-45,
    height=600,
)
last_data_fig = go.Figure(data=[tmx, tmn, ppp], layout=layout)

In [16]:
last_data_fig.show()

In [40]:
df = ds.last_data_df

In [41]:
plu_st = ["LIVES", "CHUGUR"]
df = df[["Lat", "Lon", 'TMAX']]
df = df[~df.index.isin(plu_st)]
df['size'] = 10

In [42]:
opt = {"Lat": False, "Lon": False, "size": False, "TMAX": True}
color_scale = px.colors.sequential.YlOrRd

In [44]:
mapbx = px.scatter_mapbox(
    df,
    lat="Lat",
    lon="Lon",
    hover_name=df.index,
    hover_data=opt,
    opacity=0.9,
    color='TMAX',
    color_continuous_scale=color_scale,
    size=df["size"],
    size_max=17,
    zoom=8,
    height=500,
    width=600,
)

mapbx.update_layout(
    hovermode="closest",
    autosize=False,
    mapbox=dict(
        accesstoken=ds.mapbox_token,
        bearing=0,
        center=go.layout.mapbox.Center(lat=-7.16, lon=-78.49),
        style="light",
        pitch=0,
        zoom=8,
    ),
    margin=dict(t=0, b=0, l=0, r=0),
)

In [4]:
from scripts.calculations import LastData

In [ ]:
# this_month, day, yday, 

# self.this_month = this_month
# self.day = day
# self.yday = yday

#     self.last_data_dataframe()

# def last_data_dataframe(self, this_month, day):
#     last_data = {
#         "ESTACIONES": self.mn.columns,
#         "TMIN": self.mn[
#             (self.mn.index.day == day) & (self.mn.index.month == this_month)
#         ]
#         .values[-1]
#         .tolist(),
#         "TMAX": self.mx[
#             (self.mx.index.day == day) & (self.mx.index.month == this_month)
#         ]
#         .values[-1]
#         .tolist(),
#         "PP": self.pp[
#             (self.pp.index.day == day) & (self.pp.index.month == this_month)
#         ]
#         .values[-1]
#         .tolist(),
#     }

#     self.last_data_df = pd.DataFrame(last_data)
#     self.last_data_df.set_index("ESTACIONES", inplace=True)
#     self.last_data_df = Utils(self.last_data_df, self.file).add_lat_lon()

In [5]:
ld = LastData(ds.mn, ds.mx, ds.pp, ds.this_month, ds.day, ds.yday, ds.file)

In [9]:
ds.mx[(ds.mx.index.month == ds.this_month)]

,CACHACHI,CAJABAMBA,ASUNCION,GRANJA PORCON,AUGUSTO WEBERBAUER,COSPAN,LA ENCAÑADA,JESUS,MAGDALENA,NAMORA,SAN JUAN,CELENDIN,CONTUMAZA,SAN BENITO,MONTE GRANDE,CHUGUR,SONDOR-MATARA,SAN MARCOS,QUILCATE,LLAPA,SAN MIGUEL,LIVES,SAN PABLO
Fecha,,,,,,,,,,,,,,,,,,,,,,,
2023-06-01,16.4,21.3,27.6,17.9,21.7,24.9,19.0,25.2,30.2,22.0,26.0,21.6,23.4,26.5,30.6,NaN,21.6,24.8,18.6,19.4,20.6,NaN,22.8
2023-06-02,18.4,21.3,23.2,16.9,22.0,20.8,18.7,26.8,29.6,22.3,22.8,17.7,21.4,25.2,30.4,NaN,21.1,26.0,14.8,18.0,19.6,NaN,21.2
2023-06-03,18.8,21.6,23.6,18.6,23.3,20.7,19.7,28.2,29.8,23.0,22.4,20.5,21.6,26.0,31.0,NaN,22.5,27.0,17.2,18.2,19.0,NaN,21.6
2023-06-04,18.2,21.7,25.2,19.3,22.9,21.7,20.0,28.0,30.0,23.0,26.4,21.2,22.8,24.8,31.8,NaN,21.5,26.9,19.0,20.0,20.2,NaN,23.0
2023-06-05,17.0,22.1,27.8,17.9,22.8,25.0,20.2,27.6,28.8,22.6,26.6,19.9,23.6,25.0,31.8,NaN,22.5,25.5,17.8,21.0,20.8,NaN,22.6
2023-06-06,17.2,22.7,23.6,18.7,22.2,NaN,19.6,28.2,29.0,22.3,25.4,20.0,21.6,25.6,30.0,NaN,21.2,25.6,17.6,19.6,20.0,NaN,21.6
2023-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ld.last_data_df

,Lat,Lon,TMIN,TMAX,PP
ESTACIONES,,,,,
CACHACHI,-7.45106,-78.26855,6.2,17.2,0.0
CAJABAMBA,-7.62166,-78.05131,8.9,22.7,0.0
ASUNCION,-7.32604,-78.51582,12.8,23.6,0.0
GRANJA PORCON,-7.03753,-78.63340,2.3,18.7,0.0
AUGUSTO WEBERBAUER,-7.16750,-78.49309,5.8,22.2,0.0
COSPAN,-7.42857,-78.54106,NaN,NaN,NaN
LA ENCAÑADA,-7.12327,-78.33314,2.3,19.6,0.0
JESUS,-7.24570,-78.38841,6.8,28.2,0.0
MAGDALENA,-7.25346,-78.65261,17.0,29.0,0.0
